In [ ]:
import os
import csv
from PIL import Image

class_map = {
    "stop": 0,
    "go": 1,
    "warning": 2,
    "warningLeft": 3,
    "goLeft": 4,
    "stopLeft": 5,
    "red": 6  # Optional: can be ignored if redundant with 'stop'
}

base_path = "C:/INTERNSHIP/traffic_light_detection"
dataset_path = os.path.join(base_path, "datasets/lisa")
image_root = os.path.join(base_path, "traffic_light_data/images")
label_root = os.path.join(base_path, "traffic_light_data/labels")

# Collect all annotation CSVs
csv_dirs = []
for root, _, files in os.walk(os.path.join(dataset_path, "Annotations")):
    for f in files:
        if f == "frameAnnotationsBOX.csv":
            csv_dirs.append(os.path.join(root, f))

print(f"CSV files found: {len(csv_dirs)}")

# Convert bbox to YOLO format
def convert_to_yolo(x1, y1, x2, y2, w, h):
    return (
        ((x1 + x2) / 2) / w,
        ((y1 + y2) / 2) / h,
        (x2 - x1) / w,
        (y2 - y1) / h
    )

total_labels = 0
for csv_path in csv_dirs:
    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter=';')
        for row in reader:
            label = row["Annotation tag"].strip()
            if label not in class_map:
                continue

            filename = row["Filename"].replace("\\", "/").split("/")[-1].replace(".png", ".jpg")
            x1 = int(row["Upper left corner X"])
            y1 = int(row["Upper left corner Y"])
            x2 = int(row["Lower right corner X"])
            y2 = int(row["Lower right corner Y"])

            for split in ["train", "val"]:
                img_path = os.path.join(image_root, split, filename)
                if os.path.exists(img_path):
                    try:
                        img = Image.open(img_path)
                        w, h = img.size
                        x, y, bw, bh = convert_to_yolo(x1, y1, x2, y2, w, h)
                        lbl_dir = os.path.join(label_root, split)
                        os.makedirs(lbl_dir, exist_ok=True)
                        label_file = os.path.join(lbl_dir, filename.replace(".jpg", ".txt"))
                        with open(label_file, "a") as f_lbl:
                            f_lbl.write(f"{class_map[label]} {x:.6f} {y:.6f} {bw:.6f} {bh:.6f}\n")
                        total_labels += 1
                    except Exception:
                        continue

print(f"\n YOLO labels written: {total_labels}")